## **Check Ollama Models**

In [1]:
!ollama list

NAME                     ID              SIZE      MODIFIED       
nomic-embed-text:v1.5    0a109f422b47    274 MB    14 minutes ago    
deepseek-r1:1.5b         e0979632db5a    1.1 GB    6 days ago        
llava:13b                0d0eb4d7f485    8.0 GB    2 months ago      
mistral:latest           3944fe81ec14    4.1 GB    2 months ago      


## **Setup Logger**

In [ ]:
import logging
from logging import Logger

formatter = logging.Formatter(
    '[%(levelname)-5s] %(asctime)s: %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

console_logger: Logger = logging.getLogger("console_logger")
console_logger.setLevel(logging.INFO)
console_logger.propagate = False
console_logger.handlers.clear()

if not console_logger.hasHandlers():
    console_handler = logging.StreamHandler() # Create a handler that writes log messages to the console (standard output).
    console_handler.setLevel(logging.INFO)
    console_handler.setFormatter(formatter)
    console_logger.addHandler(console_handler)

## **Setup Chain**

### **Setup Model and Prompt**

In [5]:
from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

llm = ChatOllama(model="llama3.1:8b")

template = """You are a helpful translator assistant that translates {input_language} to {output_language}.
              You are given the following {input_language} text: {text}
              You should respond with the translated text in {output_language}."""

prompt_template = PromptTemplate(
    input_variables=["input_language", "output_language", "text"],
    template=template,
)

## **Create Normal Chain**

In [ ]:
from langchain_core.output_parsers import StrOutputParser

translation_chain = prompt_template | llm | StrOutputParser() 

input1 = {
    "input_language": "English",
    "output_language": "French",
    "text": "I love programming.",
}

for chunk in translation_chain.stream(input1):
    print(chunk, end='', flush=True)

The translation of "I love programming" into French is:

"Je m'apprécie la programmation"

However, a more common and idiomatic way to express this would be:

"J'adore programmer"

This directly conveys the enthusiasm for programming rather than using "m'apprecier" which has a slightly different connotation.

## **Create Sequential Chain**

In [ ]:
summary_prompt = PromptTemplate(
    input_variables=["translated_text"],
    template="Summarize this text in one simple sentence: {translated_text}",
)


summary_sequential_chain = (
      {"translated_text": translation_chain} | # get the output of translation_chain as input
      summary_prompt |
      llm | 
      StrOutputParser()
)

text = """The sun is the closest star to Earth and is very important for life. It gives us light and warmth, which help plants grow and provide energy for all living things. Without the sun, Earth would be dark, cold, and lifeless. The sun also controls the weather and seasons, making our planet suitable to live on. That’s why it is often called the “source of life.”"""

input2 = {
    "input_language": "English",
    "output_language": "Bangla",
    "text": text,
}

for chunk in summary_sequential_chain.stream(input2):
    print(chunk, end='', flush=True)


সূর্য পৃথিবীর জন্য গুরুত্বপূর্ণ, কারণ এটি আলো ও উষ্ণতা দেয় যা সমস্ত জীবনকে সমর্থন করে।